In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [5]:
data_file_name = 'USD_JPY_data_2001-01-01_to_2020-02-10.csv'

df = pd.read_csv(data_file_name)
df.columns = ['datetime', 'volume', 'open', 'high', 'low', 'close']

df.datetime = pd.to_datetime(df.datetime)

print(df.datetime.max())

df.head()

2020-02-11 00:23:25


,datetime,volume,open,high,low,close
0,2002-05-07 20:59:35,1,127.92,127.92,127.92,127.92
1,2002-05-08 20:59:15,1,128.92,128.92,128.92,128.92
2,2002-05-09 20:59:50,1,128.38,128.38,128.38,128.38
3,2002-05-10 20:59:50,1,127.63,127.63,127.63,127.63
4,2002-05-10 22:10:00,1,127.62,127.62,127.62,127.62
